In [1]:
# Imports !

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import patsy
import statsmodels.api as sm
from scipy.stats import ttest_ind

In [2]:
# Gather data into dataframes

traffic_main_df = pd.read_csv('traffic_counts_datasd.csv')
intersections_df = pd.read_csv('Intersections.csv')

In [3]:
# Perform data cleaning on the dataframes

# Remove directional columns from traffic_counts
traffic_main_df.drop('id', 1, inplace=True)
traffic_main_df.drop('northbound_count', 1, inplace=True)
traffic_main_df.drop('southbound_count', 1, inplace=True)
traffic_main_df.drop('all_count', 1, inplace=True)
traffic_main_df.drop('eastbound_count', 1, inplace=True)
traffic_main_df.drop('westbound_count', 1, inplace=True)
traffic_main_df.drop('file_no', 1, inplace=True)
traffic_main_df.drop('count_date', 1, inplace=True)

#Drop any rows with NaN values in traffic_main counts
traffic_main_df.dropna(inplace=True) 

In [7]:
# Data cleaning on intersections data
print(intersections_df)

intersections.drop('Document ID', 1, inplace=True)
intersections.drop('Remote ID', 1, inplace=True)
intersections.drop('Document ID', 1, inplace=True)


      365c3373ce58dd5e504658f0d744bcf0  Unnamed: 1  sandiego_adt.pdf  \
0     365c3373ce58dd5e504658f0d744bcf0         NaN  sandiego_adt.pdf   
1     365c3373ce58dd5e504658f0d744bcf0         NaN  sandiego_adt.pdf   
2     365c3373ce58dd5e504658f0d744bcf0         NaN  sandiego_adt.pdf   
3     365c3373ce58dd5e504658f0d744bcf0         NaN  sandiego_adt.pdf   
4     365c3373ce58dd5e504658f0d744bcf0         NaN  sandiego_adt.pdf   
5     365c3373ce58dd5e504658f0d744bcf0         NaN  sandiego_adt.pdf   
6     365c3373ce58dd5e504658f0d744bcf0         NaN  sandiego_adt.pdf   
7     365c3373ce58dd5e504658f0d744bcf0         NaN  sandiego_adt.pdf   
8     365c3373ce58dd5e504658f0d744bcf0         NaN  sandiego_adt.pdf   
9     365c3373ce58dd5e504658f0d744bcf0         NaN  sandiego_adt.pdf   
10    365c3373ce58dd5e504658f0d744bcf0         NaN  sandiego_adt.pdf   
11    365c3373ce58dd5e504658f0d744bcf0         NaN  sandiego_adt.pdf   
12    365c3373ce58dd5e504658f0d744bcf0         NaN  sandiego_adt